In [1]:
import numpy as np
import json, sys
from scipy.stats import fisher_exact

In [2]:
from query import *
from requests import *

In [3]:
fragments_all = np.load("fragments_clust.npy")

In [4]:
chaindata = json.load(open("structures.json"))

In [5]:
chainschema = json.load(open("chainschema.json"))

In [6]:
parts = ['ph', 'sug', 'base', 'any', 'nonspecif']
states = ['ss', 'ds']
s = ['A', 'G', 'U', 'C']
seqs = [a+b+c for a in s for b in s for c in s]
m = ['A', 'C']
motifs = [a+b+c for a in m for b in m for c in m]

In [7]:
mask_redun =  np.load("redundancy-masks/1.0-seqclust-50.npy")
mask_redun[0]
fragments = fragments_all[mask_redun]   

Always do query on all fragments.
Store as many masks as possible.
To answer questions, combine the masks

Define masks

In [8]:
mask_seq = {}
for seq in seqs:
    mask_seq[seq] = (fragments["seq"] == seq.encode())

In [9]:
mask_state = {}
secstruc = query(chaindata, chainschema, fragments, [], "ss", part=None)
mask_state["ds"] = np.array(list(map(is_ds, secstruc)))
mask_state["ss"] = np.array(list(map(is_ss, secstruc)))

In [10]:
mask_close = {}
mask_veryclose = {}
for part in ["ph","sug", "base"]:    
    all_part = query(chaindata, chainschema, fragments, 99.9, "interface_protein", part=part)[:,1]
    mask_close[part] = (all_part < 99.9)
    mask_veryclose[part] = (all_part < 3.5)
mask_close["any"] = mask_close["ph"] | mask_close["sug"] | mask_close["base"]
mask_veryclose["any"] = mask_veryclose["ph"] | mask_veryclose["sug"] | mask_veryclose["base"]
mask_close["nonspecif"] = (mask_close["ph"] | mask_close["sug"]) & ( ~ mask_close["base"] )
mask_veryclose["nonspecif"] = (mask_veryclose["ph"] | mask_veryclose["sug"]) & ( ~ mask_veryclose["base"] )

In [11]:
mask_motif = {}
for motif in motifs:
    mask_motif[motif] = (fragments["motif"] == motif.encode())

# Question 1a
Do single-stranded and double-stranded trinucleotides bind protein via preferential nucleotidic parts?

In [18]:
def count_seqs(counts, seqs):
    counts_filtered = dict([(k,v) for k,v in counts.items() if k[0] in seqs])
    d = {}
    t = []
    for part in parts:
        N = {}
        percent = {}
        crosstable = []
        for state in states:
            state_any = sum([v for k,v in counts_filtered.items() if k[1] == state and k[2] == "any"])
            assert state_any > 0
            state_part = sum([v for k,v in counts_filtered.items() if k[1] == state and k[2] == part])
            state_others = state_any - state_part
            crosstable.append((state_part, state_others))
            percent[state] = 100 * state_part / state_any
            N[state] = sum([counts[m, state, part] for m in seqs])
        f = fisher_exact(crosstable)
        tt = N["ss"], N["ds"], part, percent["ss"], percent["ds"], f[0], f[1]
        t.append(tt)
        d[part] = { "N_ss": N["ss"], "N_ds": N["ds"], "ss": percent["ss"], "ds": percent["ds"], "ratio": f[0], "p-value": f[1] }
    return d, t

In [19]:
def count_mask(new_mask):
    new_count = {}
    for part in parts:
        if new_mask is None:
            mask0 = mask_close[part]
        else:
            mask0 = new_mask & mask_close[part]
        for state in states:
            mask1 = mask0 & mask_state[state]
            for seq in seqs:
                mask2 = mask1 & mask_seq[seq]
                new_count[seq, state, part] = mask2.sum()
    return new_count

In [20]:
def fisher_counts(count1, count2, part):
    N_ss1 = count1['any']["N_ss"]
    N_ss2 = count2['any']["N_ss"]
    N_ss1_part = N_ss1 * count1[part]["ss"] / 100
    N_ss2_part = N_ss2 * count2[part]["ss"] / 100
    t_ss = [ [N_ss1_part, N_ss1-N_ss1_part], [N_ss2_part, N_ss2-N_ss2_part] ]
    print(("percent_ss", count1[part]["ss"], count2[part]["ss"] ))
    print("pvalue_ss", fisher_exact(t_ss)[1])
    #
    N_ds1 = count1['any']["N_ds"]
    N_ds2 = count2['any']["N_ds"]
    N_ds1_part = N_ds1 * count1[part]["ds"] / 100
    N_ds2_part = N_ds2 * count2[part]["ds"]  / 100   
    t_ds = [ [N_ds1_part, N_ds1 - N_ds1_part], [N_ds2_part, N_ds2 - N_ds2_part] ]
    print(("percent_ds", count1[part]["ds"], count2[part]["ds"] ))
    print("pvalue_ds", fisher_exact(t_ds)[1])

In [21]:
counts = count_mask(None)

In [22]:
dd, tt = count_seqs(counts, seqs)

In [23]:
dd

{'any': {'N_ds': 2990,
  'N_ss': 7035,
  'ds': 100.0,
  'p-value': 1.0,
  'ratio': nan,
  'ss': 100.0},
 'base': {'N_ds': 1949,
  'N_ss': 5699,
  'ds': 65.18394648829431,
  'p-value': 2.4902137341051205e-62,
  'ratio': 2.2784058614428404,
  'ss': 81.00923951670221},
 'nonspecif': {'N_ds': 1041,
  'N_ss': 1336,
  'ds': 34.81605351170568,
  'p-value': 2.490213734105121e-62,
  'ratio': 0.43890336525325324,
  'ss': 18.990760483297798},
 'ph': {'N_ds': 2409,
  'N_ss': 5967,
  'ds': 80.5685618729097,
  'p-value': 2.1372516661500813e-07,
  'ratio': 1.3474855527726084,
  'ss': 84.81876332622602},
 'sug': {'N_ds': 2111,
  'N_ss': 5751,
  'ds': 70.60200668896321,
  'p-value': 5.157561242540897e-34,
  'ratio': 1.865000641942296,
  'ss': 81.74840085287846}}

In [24]:
import pandas as pd
columns = ["sequence", "N_ss", "N_ds", "part", "ss", "ds", "ratio", "pvalue"]
def table_seq(count, seqs):
    table = []
    for seq in seqs:
        _, t = count_seqs(count,[seq])
        for tt in t:
            table.append((seq,) + tt)
    df = pd.DataFrame(table, columns=columns)
    return df

In [25]:
df_all = table_seq(counts, seqs)

In [26]:
df_significant = df_all[df_all.eval('part=="base" and pvalue < 0.01')]
df_non_significant = df_all[df_all.eval('part=="base" and pvalue > 0.01')]
df_ss_base = df_all[df_all.eval('part=="base" and ss < ds')]

In [27]:
df_significant

,sequence,N_ss,N_ds,part,ss,ds,ratio,pvalue
27,AGG,84,49,base,87.500000,56.976744,5.285714,4.271624e-06
42,AUA,160,9,base,93.567251,60.000000,9.696970,7.146906e-04
52,AUU,134,13,base,88.157895,61.904762,4.581197,4.833290e-03
57,AUC,61,4,base,73.493976,26.666667,7.625000,8.088027e-04
67,ACG,32,51,base,96.969697,62.962963,18.823529,1.202880e-04
77,ACC,110,29,base,84.615385,63.043478,3.224138,3.177368e-03
87,GAG,61,46,base,81.333333,58.974359,3.031056,2.837559e-03
97,GAC,69,34,base,88.461538,56.666667,5.862745,2.662385e-05
107,GGG,73,53,base,92.405063,58.241758,8.723270,1.965936e-07
112,GGU,80,37,base,94.117647,68.518519,7.351351,8.764599e-05


In [28]:
len(df_significant)

20

In [29]:
df_ss_base

,sequence,N_ss,N_ds,part,ss,ds,ratio,pvalue
32,AGU,65,18,base,76.470588,81.818182,0.722222,0.776392
37,AGC,20,28,base,38.461538,50.000000,0.625000,0.250219
62,ACA,132,106,base,89.795918,89.830508,0.996226,1.000000
82,GAA,167,43,base,69.583333,79.629630,0.585218,0.181917
152,GCU,31,54,base,72.093023,75.000000,0.861111,0.826971
162,UAA,167,20,base,83.500000,95.238095,0.253030,0.212085
202,UUA,135,7,base,87.096774,100.000000,0.000000,0.598358
252,CAU,55,89,base,79.710145,94.680851,0.220706,0.005422
272,CGU,22,16,base,62.857143,66.666667,0.846154,0.789775
277,CGC,8,52,base,61.538462,65.822785,0.830769,0.761911


# Question 1b
Are those preferences motif (purine/pyrimidine) - dependant?

In [30]:
R = ['A', 'G']
Y = ['U', 'C']
d_YYY = count_seqs(counts, [a+b+c for a in Y for b in Y for c in Y])[0]
d_RRR = count_seqs(counts, [a+b+c for a in R for b in R for c in R])[0]

In [31]:
d_YRY = count_seqs(counts,[a+b+c for a in Y for b in R for c in Y])[0]
d_RYR = count_seqs(counts,[a+b+c for a in R for b in Y for c in R])[0]

In [32]:
d_YRY['base']

{'N_ds': 246,
 'N_ss': 563,
 'ds': 71.09826589595376,
 'p-value': 1.0399211638315497e-09,
 'ratio': 2.7909974221693434,
 'ss': 87.28682170542636}

In [33]:
fisher_counts(d_YYY, d_RRR, "nonspecif")

('percent_ss', 17.99844840961986, 20.195930670685758)
pvalue_ss 0.1637791883149834
('percent_ds', 37.724550898203596, 35.69794050343249)
pvalue_ds 0.5463915710001818


In [34]:
fisher_counts(d_RYR, d_YRY, "nonspecif")

('percent_ss', 14.361001317523057, 12.713178294573643)
pvalue_ss 0.39074181612883946
('percent_ds', 31.196581196581196, 28.90173410404624)
pvalue_ds 0.4883289695740797


# Question 1c
Are those preferences sequence - dependant? 

In [35]:
d_AAA = count_seqs(counts,["AAA"])[0]
print(d_AAA["base"])
print(d_AAA["nonspecif"])

{'ds': 77.77777777777777, 'ratio': 1.4155844155844155, 'ss': 83.20610687022901, 'N_ss': 327, 'N_ds': 7, 'p-value': 0.6523243788644034}
{'ds': 22.22222222222222, 'ratio': 0.7064220183486238, 'ss': 16.793893129770993, 'N_ss': 66, 'N_ds': 2, 'p-value': 0.6523243788644034}


In [36]:
d_GGG = count_seqs(counts,["GGG"])[0]
print("Nb_bound")
print("N_ds", d_GGG["any"]["ds"],  "N_ss", d_GGG["any"]["ss"])
print("specific")
print( "ds", d_GGG["base"]["ds"], "ss", d_GGG["base"]["ss"])
print("non-specific")
print( "ds", d_GGG["nonspecif"]["ds"], "ss", d_GGG["nonspecif"]["ss"])


Nb_bound
N_ds 100.0 N_ss 100.0
specific
ds 58.24175824175824 ss 92.40506329113924
non-specific
ds 41.75824175824176 ss 7.594936708860759


In [37]:
d_AAA

{'any': {'N_ds': 9,
  'N_ss': 393,
  'ds': 100.0,
  'p-value': 1.0,
  'ratio': nan,
  'ss': 100.0},
 'base': {'N_ds': 7,
  'N_ss': 327,
  'ds': 77.77777777777777,
  'p-value': 0.6523243788644034,
  'ratio': 1.4155844155844155,
  'ss': 83.20610687022901},
 'nonspecif': {'N_ds': 2,
  'N_ss': 66,
  'ds': 22.22222222222222,
  'p-value': 0.6523243788644034,
  'ratio': 0.7064220183486238,
  'ss': 16.793893129770993},
 'ph': {'N_ds': 9,
  'N_ss': 358,
  'ds': 100.0,
  'p-value': 1.0,
  'ratio': 0.0,
  'ss': 91.0941475826972},
 'sug': {'N_ds': 7,
  'N_ss': 351,
  'ds': 77.77777777777777,
  'p-value': 0.2574413291935603,
  'ratio': 2.3877551020408165,
  'ss': 89.31297709923665}}

In [38]:
print(counts["AAA", "ds", "base"], counts["AAA", "ds", "any"] )
print(counts["GGG", "ds", "base"], counts["GGG", "ds", "any"] )
print(sum(mask_seq["AAA"]), sum(mask_seq["GGG"]))
sum(mask_seq["AAA"] & ds), sum(mask_seq["GGG"] & ds)

7 9
53 91
1356 1043


NameError: name 'ds' is not defined

In [39]:
fisher_counts(d_AAA, d_GGG, "nonspecif")

('percent_ss', 16.793893129770993, 7.594936708860759)
pvalue_ss 0.039452799249061296
('percent_ds', 22.22222222222222, 41.75824175824176)
pvalue_ds 0.308903234739957


In [40]:
fisher_counts(d_AAA, d_GGG, "base")

('percent_ss', 83.20610687022901, 92.40506329113924)
pvalue_ss 0.039452799249061296
('percent_ds', 77.77777777777777, 58.24175824175824)
pvalue_ds 0.30890323473995707


Check where are the dsRNA AAA frag than bind via base

In [41]:
AAA_ds_base_vc = fragments[mask_seq["AAA"] & ds & mask_veryclose["base"]]
AAA_ds_base = fragments[mask_seq["AAA"] & ds & mask_close["base"]]

NameError: name 'ds' is not defined

In [42]:
m = AAA_ds_base_vc['structure']==b'6AEG'
AAA_ds_base_vc[m]

NameError: name 'AAA_ds_base_vc' is not defined

In [43]:
m = AAA_ds_base['structure']==b'6AEG'
AAA_ds_base[m]

NameError: name 'AAA_ds_base' is not defined

In [44]:
count_AAA_ds_base_per_pdb = dict(zip(*np.unique(AAA_ds_base["structure"], return_counts = True)))
count_AAA_ds_base_per_pdb

NameError: name 'AAA_ds_base' is not defined

# Question 1d
Are those differences due to protein-family properties ?

get enzyme families at:
https://enzyme.expasy.org/enzyme-byclass.html

get pdbmap from ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release

In [40]:
pdb_pfam = json.load(open("pfam.json"))
pdb_fam = json.load(open("fam.json"))

In [41]:
def inv_dict(d):
    inv_d = {}
    for k, val in d.items():
        if not isinstance(val, list):
            values = [val]
        else:
            values = val
        if len(values) == 0: continue
        for v in values:
            if v not in inv_d:
                inv_d[v] = []
            inv_d[v].append(k)
    return inv_d

In [42]:
fam_pdb = inv_dict(pdb_fam)
pfam_pdb = inv_dict(pdb_pfam)

In [43]:
%run group_fam.py
group_fam = json.load(open("group_fam.json"))
fam_group = inv_dict(group_fam)

In [44]:
a =[k for k, i in group_fam.items() if len(i) == 1]
a.sort()
[str(i) for i in a]

['1D-14L',
 'ATP-synt_ab',
 'Alba',
 'Amino_oxidase',
 'Ank',
 'Anticodon_1',
 'Btz',
 'CAT_RBD',
 'CLP1_P',
 'CPE',
 'CsrA',
 'DKCLD',
 'DNA_primase_lrg',
 'Dicer_dimer',
 'Dus',
 'FXMRP1_C_core',
 'GIIM',
 'Glu-tRNAGln',
 'HEAT_PBS',
 'Hfq',
 'HutP',
 'IPPT',
 'IgG_binding_B',
 'KOW',
 'KTI12',
 'LSM',
 'Macro',
 'MafB19-deam',
 'Myb_DNA-binding',
 'N36',
 'NHL',
 'NUDIX_2',
 'NusB',
 'OAS1_C',
 'PIN_4',
 'PUA',
 'PUF',
 'Pept_tRNA_hydro',
 'Phosphoprotein',
 'Pkinase',
 'Qua1',
 'RHD_DNA_bind',
 'RNA_GG_bind',
 'ROQ_II',
 'RRF',
 'ResIII',
 'RrnaAD',
 'SAP',
 'SBP_bac_1',
 'SHS2_Rpb7-N',
 'SLBP_RNA_bind',
 'SPOC',
 'SSB',
 'STAR_dimer',
 'SWIRM',
 'She2p',
 'SmpB',
 'Staufen_C',
 'TAL_effector',
 'TGS',
 'THUMP',
 'TRAM',
 'TROVE',
 'Tap-RNA_bind',
 'Telomerase_RBD',
 'ThiI',
 'Thrombin_light',
 'Thx',
 'Translin',
 'TrpBP',
 'Trypsin',
 'V-set',
 'Xpo1',
 'YTH',
 'dsrm',
 'tRNA_SAD']

In [50]:
print(fam_pdb["GAD"])
print(pdb_fam['3DD2'])

['4WJ4', '2D6F', '1C0A', '1EFW', '1IL2']
['1D-14L', 'Trypsin']


In [51]:
if "GFP" in fam_pdb:
    del fam_pdb['GFP']
pdb_fam['6B0B'] = [u'NAD1']

In [ ]:
def count_fam_to_group(count_per_fam):
    count_per_group = {}
    for fam, n in count_per_fam.items():
        group = fam_group[fam]
        if len(group) > 0:
            for g in group:
                if g not in count_per_group:
                    count_per_group[g] = 0
                count_per_group[g] += n
    return count_per_group

In [ ]:
def map_fam_to_group(group_fam, fam_group, fam_pdb):
    group_pdb = {}
    for fam, pdb in fam_pdb.items():
        if fam in fam_group:
            group = fam_group[fam]
            if len(group) > 0:
                for g in group:
                    if g not in group_pdb:
                        group_pdb[g] = []
                    group_pdb[g].append(pdb)
        else:
            print(fam)
    return group_pdb   

In [ ]:
def count_pdb_to_fam(count_per_pdb, pdb_pfam):  
    count_per_pfam = {}
    for pdb, n in count_per_pdb.items():
        pfams = pdb_pfam[pdb.decode()]
        if len(pfams) > 0:
            for pfam in pfams:
                if pfam not in count_per_pfam:
                    count_per_pfam[pfam] = 0
                count_per_pfam[pfam] += n
    return count_per_pfam

In [45]:
group_pdb = {}
for fam, pdbs in fam_pdb.items():
    groups = fam_group[fam]
    for group in groups:
        if group not in group_pdb:
            group_pdb[group] = []
        for pdb in pdbs:
            group_pdb[group].append(pdb)
group_pdb

{'1D-14L': ['3DD2'],
 'AAA': ['4QM6B', '4QM6A', '2XZL', '4B3G', '2XZO', '2XZL', '4B3G', '2XZO'],
 'ATP-synt_ab': ['3ICE', '5JJI', '5JJK', '5JJL'],
 'Ago': ['2HYI',
  '2XB2',
  '2J0S',
  '2J0Q',
  '3EX7',
  '4F1N',
  '5VM9',
  '4OLA',
  '4W5O',
  '4Z4D',
  '5W6V',
  '4Z4E',
  '5JS2',
  '4Z4C',
  '4KRE',
  '5T7B',
  '4W5R',
  '4Z4I',
  '6CBD',
  '4W5Q',
  '4KRF',
  '5JS1',
  '4W5T',
  '5WEA',
  '4F3T',
  '4W5N',
  '4Z4F',
  '4KXT',
  '4Z4G',
  '5KI6',
  '4Z4H',
  '4OLB',
  '5VM9',
  '4OLA',
  '4W5O',
  '4Z4D',
  '5W6V',
  '4Z4E',
  '5JS2',
  '4Z4C',
  '4KRE',
  '5T7B',
  '4W5R',
  '4Z4I',
  '6CBD',
  '4W5Q',
  '4KRF',
  '5JS1',
  '4W5T',
  '5WEA',
  '4F3T',
  '4W5N',
  '4Z4F',
  '4KXT',
  '4Z4G',
  '5KI6',
  '4Z4H',
  '4OLB',
  '5VM9',
  '4OLA',
  '4W5O',
  '4Z4D',
  '5W6V',
  '4Z4E',
  '5JS2',
  '4Z4C',
  '4KRE',
  '5T7B',
  '4W5R',
  '4Z4I',
  '6CBD',
  '4W5Q',
  '4KRF',
  '5JS1',
  '4W5T',
  '5WEA',
  '4F3T',
  '4W5N',
  '4Z4F',
  '4KXT',
  '4Z4G',
  '5KI6',
  '4Z4H',
  '4OLB',
  '4F1

In [46]:
pdb_group = inv_dict(group_pdb)
pdb_group

{'1JZY': ['Ribosom', 'Ribosom', 'Ribosom'],
 '2AKE': ['tRNA_synt'],
 '3CCQ': ['KOW',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom'],
 '4L8R': ['SLBP_RNA_bind', 'SAP', 'RNase'],
 '5XUT': ['NUC', 'CRISPR', 'CRISPR'],
 '5CCBA': ['Methyltr', 'Methyltr'],
 '3QJJ': ['Cas'],
 '2PXK': ['SRP'],
 '4KR3': ['tRNA_synt', 'tRNA_synt'],
 '1KC8': ['KOW',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'Ribosom',
  'R

check family redundancies in AAA

In [ ]:
# select AAA fragments that bind the protein
AAA = fragments[mask_seq["AAA"] & mask_close["any"]]

In [ ]:
count_AAA_per_pdb = dict(zip(*np.unique(AAA["structure"], return_counts = True)))

In [ ]:
count_AAA_per_pfam = count_pdb_to_fam(count_AAA_per_pdb, pdb_pfam)
count_AAA_per_fam = count_pdb_to_fam(count_AAA_per_pdb, pdb_fam)
count_AAA_per_group = count_fam_to_group(count_AAA_per_fam)

In [ ]:
count_AAA_per_group

In [ ]:
inv_count_AAA_per_group = inv_dict(count_AAA_per_group)
inv_count_AAA_per_group

In [ ]:
GGG = fragments[mask_seq["GGG"] & mask_close["any"]]
count_GGG_per_pdb = dict(zip(*np.unique(GGG["structure"], return_counts = True)))
count_GGG_per_fam = count_pdb_to_fam(count_GGG_per_pdb, pdb_fam)
count_GGG_per_group = count_fam_to_group(count_GGG_per_fam)
inv_count_GGG_per_group = inv_dict(count_GGG_per_group)

In [ ]:
inv_count_GGG_per_group

Do the stats change if we remove Ribosomal complexes?

In [ ]:
nn = np.array(group_pdb['Ribosom'])
len(nn)

In [ ]:
fragments["structure"]

In [ ]:
mask_noRib = np.isin(fragments["structure"], group_pdb['Ribosom'], invert = True)
mask_Rib = np.isin(fragments["structure"], group_pdb['Ribosom'])

In [ ]:
print(sum(mask_noRib))
print(sum(mask_Rib))
392112/(392112+91006)

In [ ]:
AAA_noRib = fragments[mask_seq["AAA"] & mask_noRib ]
AAA_Rib = fragments[mask_seq["AAA"] & mask_Rib]
GGG_noRib = fragments[mask_seq["GGG"] & mask_noRib]
GGG_Rib = fragments[mask_seq["GGG"] & mask_Rib]

In [ ]:
print(len(fragments[mask_seq["GGG"] & mask_Rib & mask_state['ds']]))
print(len(fragments[mask_seq["GGG"] & mask_noRib & mask_state['ds']]))

In [ ]:
for i in [AAA_noRib, AAA_Rib, GGG_noRib, GGG_Rib]:
    print(len(i))

In [ ]:
def count_mask(new_mask):
    new_count = {}
    for part in parts:
        if new_mask is None:
            mask0 = mask_close[part]
        else:
            mask0 = new_mask & mask_close[part]
        for state in states:
            mask1 = mask0 & mask_state[state]
            for seq in seqs:
                mask2 = mask1 & mask_seq[seq]
                new_count[seq, state, part] = mask2.sum()
    return new_count

In [ ]:
count_noRib = count_mask(mask_noRib)

In [ ]:
count_all = counts
np.isin(fragments["structure"], pfam_pdb[pfam])

In [ ]:
mask_all = np.isin(fragments["seq"], seqs)

In [ ]:
print(count_noRib["AAA", 'ds', 'base'])
print(counts["AAA", 'ds', 'base'])

In [ ]:
d_noRib_AAA = count_seqs(count_noRib, ["AAA"])[0]
d_noRib_GGG = count_seqs(count_noRib, ["GGG"])[0]
d_AAA = count_seqs(counts, ["AAA"])[0]
d_GGG = count_seqs(counts, ["GGG"])[0]

In [ ]:
fisher_counts(d_02A_AAA, d_02A_GGG, "nonspecif")

In [ ]:
fisher_counts(d_02A_AAA, d_02A_GGG, "base")

In [ ]:
d_02A_AAA['any']

In [ ]:
d_noRib_GGG['any']

In [ ]:
fisher_counts(d_02A_AAA, d_02A_GGG, "specif")

In [ ]:
print(0.8947 * 76)
print(0.70065 * 922)
print(0.5086 * 1111)
print(0.8766 * 235)

In [ ]:
tt = [[276, 922-276],[29,235-29]]
fisher_exact(tt)

In [ ]:
fisher_counts(d_noRib_AAA, d_noRib_GGG, "nonspecif")

In [ ]:
print(0.9015 * 76)
print(0.96795 * 235)
print(0.67 * 1111)
print(0.8527 * 922)

In [ ]:
print(0.9015 * 71)
print(0.96795 * 156)
print(0.67 * 315)
print(0.8527 * 509)

In [ ]:
tt = [[64, 71-64],[434,509-434]]
fisher_exact(tt)

# Question 2
For single-stranded fragments with a protein-bound middle residue,
 how many stacking interactions does it make with neighboring bases?

In [ ]:
ph = mask_veryclose["ph"]
base = mask_veryclose["base"]
sug = mask_veryclose["sug"]
nonspecif = mask_veryclose["nonspecif"]
any = mask_veryclose["any"]

In [ ]:
# mask on frag_ss with central base stacking with base n-1 / n+1 / not neighbor
ss_stackprev = query(chaindata, chainschema, fragments, 0, "stacking", pos="n-1")
ss_stacknext = query(chaindata, chainschema, fragments, 0, "stacking", pos="n+1")
ss_stackother = query(chaindata, chainschema, fragments, 0, "stacking", pos="other")

In [ ]:
stackprev = ss_stackprev[:,1]
stacknext = ss_stacknext[:,1]
stackother = ss_stackother[:,1]
nostack = ~stackprev & ~stacknext & ~stackother

In [ ]:
sum(any & ~base & ss & stackother)

In [ ]:
sum(sug & ss & stackother & (stackprev|stacknext))

In [ ]:
sum(any & ss)

In [ ]:
def pp(part, name, value, tot):
    print(part, name, value, round(100 * float(value)/tot))

for part in parts:
    m = mask_veryclose[part] & mask_state['ss']
    tot = sum(m)
    print(tot)
    pp(part, "none", sum(m&~(stackprev|stacknext|stackother)), tot)
    pp(part, "any_neighbor", sum(m&(stackprev|stacknext)), tot)
    pp(part, "both_neighbor", sum(m&(stackprev&stacknext)), tot)
    pp(part, "other", sum(m&stackother), tot)

In [ ]:
a1 = sum(any & mask_state['ss'] & (stackprev|stacknext|stackother))
b1 = sum(any & mask_state['ss'] & ~(stackprev|stacknext|stackother))
a2 = sum(base & mask_state['ss'] & (stackprev|stacknext|stackother))
b2 = sum(base & mask_state['ss'] & ~(stackprev|stacknext|stackother))
table = [[a1, b1],  [a2, b2]]
fisher_exact(table)

In [ ]:
def pval(tot1, n1, tot2, n2):
    table = [[n1, tot1-n1], [n2, tot2-n2]]
    r, p = fisher_exact(table)
    return p
    
count_seq_nostacking = {}
mask_ss = mask_state['ss']
mask_base = mask_veryclose["base"]
mask_phsug = mask_veryclose["nonspecif"]
for seq in seqs:
    m_base = mask_base & mask_seq[seq]
    m_phsug = mask_phsug &  mask_seq[seq]
    tot_base = sum(m_base)
    tot_phsug = sum(m_phsug)
    nostack_base = sum(nostack & m_base)
    nostack_phsug = sum(nostack & m_phsug)
    p_base = round( 100 * nostack_base/tot_base)
    p_phsug = round( 100 * nostack_phsug/tot_phsug)
    p = pval(tot_base, nostack_base, tot_phsug, nostack_phsug)
    count_seq_nostacking[seq] = [p_base, p_phsug, p]

In [ ]:
count_seq_nostacking["AAA"]

In [ ]:
signif = [k for k in seqs if count_seq_nostacking[k][2]< 0.001]
not_signif = [k for k in seqs if count_seq_nostacking[k][2] > 0.001]

In [ ]:
signif

In [ ]:
mask_phsug = mask_nonspecif

In [ ]:
count_motif_nostacking = {}
for m in motifs:
    m_base = mask_base & mask_motif[m]
    m_phsug = mask_phsug &  mask_motif[m]
    tot_base = sum(m_base)
    tot_phsug = sum(m_phsug)
    nostack_base = sum(nostack & m_base)
    nostack_phsug = sum(nostack & m_phsug)
    p_base = round( 100 * nostack_base/tot_base)
    p_phsug = round( 100 * nostack_phsug/tot_phsug)
    p = pval(tot_base, nostack_base, tot_phsug, nostack_phsug)
    count_motif_nostacking[m] = [p_base, p_phsug, p]

In [ ]:
signif = [k for k in motifs if count_motif_nostacking[k][2]< 0.001]
not_signif = [k for k in motifs if count_motif_nostacking[k][2] > 0.001]

In [ ]:
signif

In [ ]:
not_signif

# Question 3
Does the original sequence have an influence on the structure of the mutated fragment?
=> do the frag with same original sequence cluster together?

In [45]:
def frag2dict(f):
    return {field:f[field] for field in fragments.dtype.fields}

In [46]:
frag2dict(fragments[1])

{'chain': b'0',
 'clust0.2': 2247,
 'clust0.2_center': False,
 'clust1.0': 1,
 'clust1.0_center': False,
 'clust2.0': 1,
 'clust2.0_center': False,
 'frag': 17170,
 'indices': array([2413, 2414, 2415], dtype=uint32),
 'missing_atoms': array([0, 0, 0], dtype=uint8),
 'model': 1,
 'motif': b'AAA',
 'resid': array([b'2522', b'2523', b'2524'], dtype='|S5'),
 'seq': b'GGG',
 'structure': b'1OND'}

In [14]:
# sequences belonging to each motif
motif_seq = {}
for m in motifs:
    seq = [f["seq"].decode() for f in fragments[mask_motif[m]]]
    sequ = list(set(seq))
    motif_seq[m] = sequ
motif_seq

{'AAA': ['AGG', 'GAA', 'AAG', 'AAA', 'GGA', 'GAG', 'AGA', 'GGG'],
 'AAC': ['GGU', 'AAC', 'GGC', 'GAC', 'GAU', 'AAU', 'AGC', 'AGU'],
 'ACA': ['ACG', 'GUG', 'GUA', 'GCA', 'GCG', 'AUG', 'ACA', 'AUA'],
 'ACC': ['ACC', 'ACU', 'AUU', 'GUC', 'GCC', 'GUU', 'GCU', 'AUC'],
 'CAA': ['CGG', 'UGG', 'UAA', 'CAG', 'UAG', 'CAA', 'CGA', 'UGA'],
 'CAC': ['UAU', 'CGU', 'UGC', 'CAC', 'CGC', 'UGU', 'UAC', 'CAU'],
 'CCA': ['CUA', 'CCA', 'UCA', 'CCG', 'UUG', 'UUA', 'UCG', 'CUG'],
 'CCC': ['CCC', 'CUC', 'CCU', 'UCC', 'UCU', 'UUU', 'CUU', 'UUC']}

In [13]:
motifs

['AAA', 'AAC', 'ACA', 'ACC', 'CAA', 'CAC', 'CCA', 'CCC']

In [12]:
# Number of clusters per motif
motif_Nclust1 = {}
for motif in motifs:
    clust1 = [f["clust1.0"] for f in fragments[mask_motif[motif]]]
    clust1u = len(np.unique(clust1))
    motif_Nclust1[motif] = clust1u
motif_Nclust1

{'AAA': 4670,
 'AAC': 3369,
 'ACA': 3120,
 'ACC': 2565,
 'CAA': 3511,
 'CAC': 2433,
 'CCA': 2821,
 'CCC': 3091}

In [15]:
# Number of frag of each sequences in each cluster
count_clust_seq = {}
for m in motifs:
    count_clust_seq[m] = {}
    frag_motif = fragments[mask_motif[m]]
    for clust1 in range(motif_Nclust1[m]):
        count_clust_seq[m][str(clust1)] = {}
        frag_clust = frag_motif[(frag_motif["clust1.0"] == clust1)]
        for seq in motif_seq[m]:
            n = (frag_clust["seq"] == seq.encode()).sum()
            count_clust_seq[m][str(clust1)][seq] = n
#            
# Number of sequences in each cluster
count_clust_seq_nonzero = {}
for m in motifs:
    count_clust_seq_nonzero[m] = {}
    d = count_clust_seq[m]
    for clust in d:
        count_clust_seq_nonzero[m][clust] = len([s for s in d[clust] if d[clust][s] > 0])

In [16]:
# Nomber of clusters containing 1, 2 .. 8 sequences
count_Nclust_per_Nseq = {}
for seq in count_clust_seq_nonzero:
    count_Nclust_per_Nseq[seq] = {}
    for i in range(1,9):
        count_Nclust_per_Nseq[seq][i] = 0
    for k in count_clust_seq_nonzero[seq]:
        l = count_clust_seq_nonzero[seq][k]
        count_Nclust_per_Nseq[seq][l] += 1
count_Nclust_per_Nseq

{'AAA': {1: 4321, 2: 210, 3: 73, 4: 25, 5: 19, 6: 8, 7: 9, 8: 5},
 'AAC': {1: 3081, 2: 170, 3: 65, 4: 22, 5: 19, 6: 3, 7: 3, 8: 6},
 'ACA': {1: 2886, 2: 147, 3: 39, 4: 21, 5: 12, 6: 6, 7: 4, 8: 5},
 'ACC': {1: 2328, 2: 143, 3: 48, 4: 24, 5: 7, 6: 4, 7: 7, 8: 4},
 'CAA': {1: 3194, 2: 206, 3: 53, 4: 21, 5: 20, 6: 6, 7: 4, 8: 7},
 'CAC': {1: 2242, 2: 115, 3: 43, 4: 16, 5: 8, 6: 2, 7: 4, 8: 3},
 'CCA': {1: 2599, 2: 135, 3: 46, 4: 16, 5: 11, 6: 5, 7: 4, 8: 5},
 'CCC': {1: 2865, 2: 127, 3: 51, 4: 23, 5: 8, 6: 4, 7: 5, 8: 8}}

In [17]:
#frequence of each sequence in the non-redundant fragments of each motif
frag_freq_per_seq = {}
for m in motifs:
    N_motif = sum(mask_motif[m]) 
    frag_freq_per_seq[m] = {}
    for seq in motif_seq[m]:
        g = mask_seq[seq]
        freq = float((mask_seq[seq] & mask_motif[m]).sum()) / N_motif
        frag_freq_per_seq[m][seq] = freq

In [18]:
# assign cluster size to each fragment
motif_names, motif_indices = np.unique(fragments["motif"], return_inverse=True)
cluster_indices = fragments["clust1.0"] + 100000 * motif_indices
clust_ids0, clust_sizes0 = np.unique(cluster_indices, return_counts=True)

ordered_clust_sizes = np.zeros(max(clust_ids0)+1, dtype=int)
ordered_clust_sizes[clust_ids0] = clust_sizes0
frag_clust_size = ordered_clust_sizes[cluster_indices]

Compare frequence of a seq in all fragments to frequence in singleton clusters => does that sequence have a larger conformatinoal diversity than other sequence in the same motif?

In [19]:
#frequence of each sequence in the singletons
mask_clust_size_1 = (frag_clust_size == 1)
single_freq = {}
for motif in motifs:
    single_freq[motif] = {}
    N_single = float((mask_motif[motif] & mask_clust_size_1).sum())
    assert N_single > 0, motif
    for seq in motif_seq[motif]:
        single_freq[motif][seq] = (mask_seq[seq] & mask_clust_size_1).sum() / N_single

In [20]:
print(frag_freq_per_seq["AAA"]["AAA"])
print(single_freq["AAA"]["AAA"])

0.13901852637533588
0.14418721690991446


In [21]:
print(frag_freq_per_seq["AAA"]["GGG"])
print(single_freq["AAA"]["GGG"])

0.10210719841606562
0.08958228485153498


In [22]:
print(frag_freq_per_seq["CCC"]["CCC"])
print(single_freq["CCC"]["CCC"])

0.10083493898522801
0.0737913486005089


In [23]:
print(frag_freq_per_seq["CCC"]["UUU"])
print(single_freq["CCC"]["UUU"])

0.21815457075572683
0.2529989094874591


Compare frequence of a seq in all fragments to average frequence in clusters of a given size (> 8*5 = 40)
=> does that sequence have a tendency to cluster together ?

In [88]:
fragments[0]["clust1.0"]
(fragments["clust1.0"] == 92).sum()
frags = fragments[(fragments["clust1.0"] == 92)]
frags

array([(b'AAA', 66603, b'484D', 17, b'B', [b'10', b'11', b'12'], [  10,   11,   12], [0, 0, 0], b'GAG',  8511,  True, 92,  True, 1, False),
       (b'ACA',  2885, b'1I94',  1, b'A', [b'1208', b'1209', b'1210'], [1207, 1208, 1209], [0, 0, 0], b'ACA',  8021,  True, 92,  True, 1, False),
       (b'ACA', 51120, b'4JI7B',  1, b'A', [b'1227', b'1228', b'1229'], [1201, 1202, 1203], [0, 0, 0], b'ACA', 12744, False, 92, False, 1, False),
       (b'ACA', 50926, b'4JI7A',  1, b'A', [b'1227', b'1228', b'1229'], [1201, 1202, 1203], [0, 0, 0], b'ACA', 12744,  True, 92, False, 1, False),
       (b'CCC', 32925, b'3ICE',  1, b'G', [b'4', b'5', b'6'], [   4,    5,    6], [0, 0, 0], b'UUU',  6453,  True, 92,  True, 1, False),
       (b'ACC',  4886, b'1K01',  1, b'A', [b'2750', b'2751', b'2752'], [2650, 2651, 2652], [0, 0, 0], b'GCC', 10311, False, 92, False, 1, False),
       (b'ACC', 22564, b'2D3O',  1, b'0', [b'2290', b'2291', b'2292'], [2215, 2216, 2217], [0, 0, 0], b'AUC', 10966,  True, 92, False, 1,

In [24]:
# assign cluster size to each fragment, per motif
frag_clust_size = {}
clust_sizes = {}
clust_ids = {}
for motif in motifs:
    frags = fragments[mask_motif[motif]]
    cluster_indices = frags["clust1.0"]
    clust_ids[motif], clust_sizes[motif] = np.unique(cluster_indices, return_counts=True)
    #ordered_clust_sizes = np.zeros(max(clust_ids[motif])+1, dtype=int)
    #ordered_clust_sizes[clust_ids[motif]] = clust_sizes[motif]
    #frag_clust_size[motif] = ordered_clust_sizes[cluster_indices]

In [25]:
#occurences of each sequence in the non-redundant fragments of each motif
n_seq = {}
for motif in motifs:
    for seq in motif_seq[motif]:
        n_seq[seq] = mask_seq[seq].sum()

In [26]:
# Occurences of each motif
n_motif = {}
for m in motifs:
    n_motif[m] = mask_motif[m].sum()

In [27]:
# Indices of clusters of size >= x, per motif
bigclust = {}
bigclust["40"] = {m:list(clust_ids[m][clust_sizes[m] >= 40]) for m in motifs}
bigclust["30"] = {m:list(clust_ids[m][clust_sizes[m] >= 30]) for m in motifs}
bigclust["10"] = {m:list(clust_ids[m][clust_sizes[m] >= 10]) for m in motifs}

In [28]:
sum([len(bigclust["10"][m]) for m in motifs])

205

In [31]:
[sum((fragments["clust1.0"] == i) & mask_motif["AAA"]) for i in bigclust["30"]["AAA"][:10] ]

[270, 58, 69, 46, 33, 53, 31, 46, 31, 44]

In [29]:
# occurences of each sequence in each cluster of size >= x
t = {}
freq_obs = {}
#for m in motifs:
for m in motifs:
    freq_obs[m] = {}
    t[m] = {}
    for i in bigclust["10"][m]:
        freq_obs[m][i] = {}
        mask0 = (fragments["clust1.0"] == i) & mask_motif[m]
        t[m][i] = mask0.sum()
        #print("***************************", i)
        #print(m, t[m][i])
        for s in motif_seq[m]:
            mask1 = mask0 & (fragments["seq"] == s.encode())
            freq_obs[m][i][s] = mask1.sum()
            #print(s, sum(mask1))

In [30]:
# Nb "fails" in non-redundant frag of each motif for each seq
n_nonseq = {}
freq_e = {}
for m in motifs:
    seqs = motif_seq[m]
    for s in seqs:
        n_nonseq[s] = n_motif[m] - n_seq[s]
        freq_e[s] = int(round(100 * n_seq[s] / float(n_motif[m])))

In [31]:
# enrichment of each cluster in each sequence, with p-value
clusters_enrichment = {}
for m in motifs:
    clusters_enrichment[m] = {}
    print("   motif %s"%m)
    for i in bigclust["10"][m]:
        seqs = motif_seq[m]
        clusters_enrichment[m][i] = {}
        for s in seqs:
            nc_seq = freq_obs[m][i][s]
            nc_nonseq = t[m][i] - freq_obs[m][i][s]
            freq_o = float(freq_obs[m][i][s]) / t[m][i]
            table = [[n_seq[s], n_nonseq[s]], [nc_seq, nc_nonseq]]
            f, p = fisher_exact(table)
            clusters_enrichment[m][i][s] = [freq_o, p]
            #if p < 0.05:
            #    print("%s, clust %.3f%%, tot %i%%, p-val %.3f"%(s, freq_o, freq_e[s] , p))

   motif AAA
   motif AAC
   motif ACA
   motif ACC
   motif CAA
   motif CAC
   motif CCA
   motif CCC


In [32]:
m = "CCC"
[[s, clusters_enrichment[m][i][s]] for s in motif_seq[m]]

[['CCC', [0.0, 0.6163090680180277]],
 ['CUC', [0.09090909090909091, 1.0]],
 ['CCU', [0.0, 0.6150481595141855]],
 ['UCC', [0.0, 0.6291084910594509]],
 ['UCU', [0.2727272727272727, 0.08969827290518134]],
 ['UUU', [0.45454545454545453, 0.07046783451873184]],
 ['CUU', [0.18181818181818182, 0.3388614715042333]],
 ['UUC', [0.0, 0.3789357447696692]]]

In [33]:
#Nb of cluster significantly enriched in only x sequences
x = 0
tot = 0
for m in motifs:
    nc = 0
    e_list = []
    for i in bigclust["10"][m]:
        ne = 0
        for s in motif_seq[m]:
            f, p = clusters_enrichment[m][i][s]
            if p < 0.05 and freq_e[s] < 100 * f: 
                ne +=1
                e_list.append(s)
        if ne == x:
            nc += 1
    tot +=nc
    #print(m, nc, set(e_list), len(clust_sizes40[m]))
    print(m, nc, len(bigclust["10"][m]))    
print(tot)

AAA 9 40
AAC 7 27
ACA 3 22
ACC 4 21
CAA 6 32
CAC 6 16
CCA 10 24
CCC 8 23
53


In [138]:
#Nb of clusters that do contain the original sequence A/C-A/C-A/C
for m in motifs:
    n_clust_no_seq = 0
    for i in bigclust["10"][m]:
        if freq_obs[m][i][m] > 0: 
            n_clust_no_seq +=1
    print(m, n_clust_no_seq, len(bigclust["10"][m]))

AAA 57 138
AAC 46 114
ACA 37 97
ACC 31 70
CAA 55 146
CAC 27 70
CCA 35 74
CCC 28 59


In [34]:
#Nb of clusters that contain x ori sequence
x = 0
for m in motifs:
    nc = 0
    for i in bigclust["10"][m]:
        ns = 0
        s_list = []
        for s in motif_seq[m]:
            if freq_obs[m][i][s] > 0: 
                ns +=1
                s_list.append(s)
        if ns == x:
            nc += 1
            print(m, nc, s_list, len(clust_sizes40[m]))

In [35]:
#Nb of clusters with x sequences, for x in [1,8], for each motif
tot = 0
clustid_nseq = {}
for m in motifs:
    clustid_nseq[m] = {}
    n_x = {}
    for x in range(1,9):
        clustid_nseq[m][x] = []
        n_x[x] = 0
        for i in bigclust["10"][m]:
            ns = 0
            for s in motif_seq[m]:
                if freq_obs[m][i][s] > 0: 
                    ns +=1
            if ns == x:
                n_x[x] += 1
                clustid_nseq[m][x].append(i)
        tot += n_x[x]
    print(m, n_x)
print(tot)

AAA {1: 0, 2: 0, 3: 8, 4: 5, 5: 6, 6: 7, 7: 9, 8: 5}
AAC {1: 0, 2: 0, 3: 4, 4: 1, 5: 11, 6: 3, 7: 2, 8: 6}
ACA {1: 0, 2: 1, 3: 1, 4: 4, 5: 4, 6: 3, 7: 4, 8: 5}
ACC {1: 0, 2: 0, 3: 2, 4: 4, 5: 1, 6: 3, 7: 7, 8: 4}
CAA {1: 1, 2: 0, 3: 3, 4: 2, 5: 11, 6: 4, 7: 4, 8: 7}
CAC {1: 0, 2: 0, 3: 2, 4: 3, 5: 2, 6: 2, 7: 4, 8: 3}
CCA {1: 0, 2: 1, 3: 0, 4: 3, 5: 7, 6: 4, 7: 4, 8: 5}
CCC {1: 0, 2: 0, 3: 2, 4: 4, 5: 2, 6: 2, 7: 5, 8: 8}
205


In [36]:
clustid_nseq["ACA"][2]

[229]

In [59]:
m="ACA"
clustidpur = clustid_nseq[m][2]
mask_clustpur = mask_motif[m] & (fragments["clust1.0"] == clustidpur)
fragpur = fragments[mask_clustpur]
print(len(fragpur))
struc = fragpur["structure"]
chain = fragpur["chain"]
for i in range(len(chain)):
    print(struc[i].decode(), chain[i].decode())
pfam = [chaindata[s.decode()]["pfam"] for s in struc]
#pfam

12
5NRGA X
1HNZ A
5DDP B
1FJE A
2XD0 I
2HGH B
1K01 A
4ATO G
1JBS D
2AAR 0
4LCK F
3CCJ 0


In [38]:
for f in fragpur:
    print(f["structure"], [int(k) for k in f["resid"]])
    

b'5NRGA' [904, 905, 906]
b'1HNZ' [890, 891, 892]
b'5DDP' [26, 27, 28]
b'1FJE' [16, 17, 18]
b'2XD0' [21, 22, 23]
b'2HGH' [26, 27, 28]
b'1K01' [469, 470, 471]
b'4ATO' [23, 24, 25]
b'1JBS' [10, 11, 12]
b'2AAR' [1279, 1280, 1281]
b'4LCK' [20, 21, 22]
b'3CCJ' [464, 465, 466]


In [48]:
fragpur["clust2.0"]

array([12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12], dtype=uint16)

In [49]:
frag2A = mask_motif[m] & (fragments["clust2.0"] == 12 )

In [50]:
frag2A.sum()

50

In [51]:
frag2A.sum()
frags = fragments[frag2A]
print(frags["seq"])

[b'GUA' b'GUA' b'ACA' b'GCA' b'GUA' b'GUA' b'GUA' b'GCG' b'AUA' b'GUG'
 b'GUA' b'GUA' b'GUA' b'AUA' b'GUA' b'GUA' b'GCG' b'GUA' b'GUG' b'ACA'
 b'GUA' b'GUA' b'GUG' b'GCG' b'GUA' b'GUG' b'AUA' b'GUA' b'GUA' b'GUA'
 b'GUA' b'GUA' b'ACG' b'GUA' b'GUA' b'GCA' b'GUA' b'GUA' b'GUA' b'AUA'
 b'GUA' b'GUA' b'GUA' b'AUA' b'GUA' b'GCG' b'GUA' b'AUA' b'GUA' b'GUA']


In [52]:
np.unique(frags["seq"], return_counts=True)

(array([b'ACA', b'ACG', b'AUA', b'GCA', b'GCG', b'GUA', b'GUG'],
       dtype='|S3'), array([ 2,  1,  6,  2,  4, 31,  4]))

In [53]:
print( [i.decode() for i in fragpur["structure"]])

['5NRGA', '1HNZ', '5DDP', '1FJE', '2XD0', '2HGH', '1K01', '4ATO', '1JBS', '2AAR', '4LCK', '3CCJ']


In [54]:
[ set(pdb_group.get(j.decode(), [])) for j in struc ]

[{'KOW', 'Ribosom'},
 {'KH', 'Ribosom', 'Thx'},
 {'RRM'},
 {'RRM'},
 {'toxin'},
 {'zf'},
 {'Ribosom'},
 {'toxin'},
 set(),
 {'RIG', 'Ribosom', 'Trigger'},
 {'Ribosom'},
 {'KOW', 'Ribosom'}]

In [64]:
# proba of each cluster to be pur (all frag from same seq)
prob_pur = np.zeros(800)
count = 0
from math import factorial as fact
from math import log, exp
proba_pur = {}
for m in motifs:
    proba_pur[m] = {}
    for i in bigclust["10"][m]:
        n = mask_motif[m].sum() # Nb frag of motif m
        k = t[m][i] # cluster size
        #print(n, k)
        log_N_any = log(fact(n)) - log(fact(n-k))
        N_pur = 0
        for s in motif_seq[m]:
            ns = n_seq[s]
            #print(ns, k)
            if ns > k:
                nn = 1
                for j in range(ns-k+1, ns):
                    nn *= j
                N_pur += nn
        #print(N_pur)
        if N_pur == 0:
            proba_pur[m][i] = 0
        else:
            log_proba = log(N_pur) - log_N_any
            proba_pur[m][i] = exp(log_proba) 
            prob_pur[count] = proba_pur[m][i]
            count += 1

In [65]:
# expected number of pur clusters for a random clustering
prob_pur.sum()

1.0035907156202207e-09

In [94]:
# Pur clusters
pur_clust = {}
for m in motifs:
    pur_clust[m] = []
    for i in clust_sizes40[m]:
        ns = 0
        for s in motif_seq[m]:
            if freq_obs[m][i][s] > 0: 
                ns +=1
        if ns == 1:
            pur_clust[m].append(i)   

In [95]:
 pur_clust["AAA"]

[1094]

In [ ]:
# seq50 in each pur_clust
fams_pur_clust = {}
seq50_pur_clust = {}
for m in motifs:
    fams_pur_clust[m] = {}
    seq50_pur_clust[m] = {}
    for i in pur_clust[m]:
        mask0 = mask_motif[m] & (fragments["clust1.0"] == i)
        strucs = fragments[mask0]["structure"]
        ss = [s.decode() for s in list(set(strucs))]
        d = chaindata[s]
        chains = d["protchains"]
        s50 = [d["seqclust"][c]["50"] for c in chains]
        #fams_pur_clust[m][i] = [chaindata[s]["pfam"] for s in ss]
        seq50_pur_clust[m][i] = np.array(s50)
        print(m, i, strucs)
        print(s50)
        print(len(s50), len(np.unique(s50)))

In [103]:
chaindata["1B7F"]["seqclust"]

{'A': {'100': 47245,
  '30': 3990,
  '40': 5704,
  '50': 10567,
  '70': 11534,
  '90': 12056,
  '95': 12143},
 'B': {'100': 47245,
  '30': 3990,
  '40': 5704,
  '50': 10567,
  '70': 11534,
  '90': 12056,
  '95': 12143}}

In [101]:
m="AAA"
i=1094
mask0 = mask_motif[m] & (fragments["clust1.0"] == i)
print(mask0.sum())
strucs = [s.decode() for s in fragments[mask0]["structure"]]
seq50_pur = []
for s in strucs:
    d = chaindata[s]
    chains = d["protchains"]
    print(s, chains)
    s50 = [d["seqclust"][c]["50"] for c in chains]
    for j in s50:
        seq50_pur.append(j)

53
2RSK ['C', 'D']


KeyError: '50'

In [ ]:
mask0 = mask_motif["AAA"] & (fragments["clust1.0"] == 700)

In [ ]:
fams_pur_clust["AAA"].keys()

# Question 4
Does the bound protein have an influence on the structure of the mutated fragment?
=> do the frag bound to same prot family cluster together?

# Question 5
=> do the ssRNA and dsRNA cluster together?

In [ ]:
from matplotlib import pyplot as plt
#frags = fragments[::1000] ###
#print(frags["motif"])
motif_names, motif_indices = np.unique(fragments["motif"], return_inverse=True)
cluster_indices = fragments["clust1.0"] + 100000 * motif_indices
#print(cluster_indices)
clust_ids, clust_sizes = np.unique(cluster_indices, return_counts=True)
#print(clust_sizes)
print("Clusters:", len(clust_sizes))
print("10 largest clusters:", np.sort(clust_sizes)[::-1][:10])
bins = (np.arange(10)+1).tolist() + [20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000, len(fragments)]

hist = np.histogram(clust_sizes, bins=bins)
import pandas as pd
print(pd.DataFrame([bins, hist[0]]).transpose())

fig, ax = plt.subplots()
x=np.arange(len(hist[0]))
ax.bar(height=hist[0], x=x, log=True)
ax.set_xticks(np.arange(len(bins)))
ax.set_xticklabels(bins, rotation='vertical')
plt.show()
fig, ax = plt.subplots()
ax.bar(height=hist[0], x=x)
ax.set_xticks(np.arange(len(bins)))
ax.set_xticklabels(bins, rotation='vertical')
plt.show()

In [ ]:
largest = sum(np.sort(clust_sizes)[::-1][:8])
print(largest, len(fragments), 100 * largest/len(fragments))

In [ ]:
clust_ids[np.argsort(clust_sizes)[::-1][:20]]

In [ ]:
mask_clust1 = (fragments["clust1.0"] == 1)

In [ ]:
mask_clust1.sum() / len(fragments)

In [ ]:
sum(mask_state["ds"] &  mask_clust1)

In [ ]:
sum(mask_state["ds"])

In [ ]:
sum(mask_clust1)

In [ ]:
mask_clust2 = (fragments["clust1.0"] == 2)

In [ ]:
sum(mask_state["ds"] &  mask_clust2)

In [ ]:
motif_names

In [ ]:
print(clust_sizes[:10], clust_ids[:10])
print(clust_ids[:4672])
print(motif_names)
ordered_clust_sizes = np.zeros(max(clust_ids)+1, dtype=int)
ordered_clust_sizes[clust_ids] = clust_sizes
ordered_clust_sizes[100001], clust_sizes[4671], clust_ids[4671]
frag_clust_size = ordered_clust_sizes[cluster_indices]
#fragments[100000]["clust1.0"], fragments[100000]["motif"], ordered_clust_sizes[500001], frag_clust_size[100000]
mask_clust_size_2 = (frag_clust_size == 2)

In [ ]:
mask_size2_ss = mask_clust_size_2 & mask_state["ss"]
mask_size2_ds = mask_clust_size_2 & mask_state["ds"]

In [ ]:
sum(mask_size2_ss), sum(mask_size2_ds)

In [ ]:
sum(mask_clust_size_2)

In [ ]:
clust_size2 = list(clust_ids[clust_sizes==2])

dict_clust_size2 = {k:{"ss": 0, "ds":0} for k in clust_size2 }
for state in ["ds", "ss"]:
    frag_clustsize2 = fragments[mask_clust_size_2 & mask_state[state]]
    for frag in frag_clustsize2:
        motif = frag["motif"]
        motif_ind = list(motif_names).index(motif)
        clust_global_ind = frag["clust1.0"] + 100000 * motif_ind
        #if clust_global_ind not in dict_clust_size2:
        #    dict_clust_size2[clust_global_ind] = {"ss": 0, "ds":0}
        dict_clust_size2[clust_global_ind][state] += 1
    

In [ ]:
dict_clust_size2

In [ ]:
ss = np.array([v["ss"] for v in dict_clust_size2.values()])
ss0 = (ss == 0).sum()
ss1 = (ss == 1).sum()
ss2 = (ss == 2).sum()
print(ss0/len(ss), ss1/len(ss), ss2/len(ss))


In [ ]:
ssfreq = sum(mask_size2_ss)/sum(mask_clust_size_2)
print(ssfreq)
print((1-ssfreq)*(1-ssfreq), 2*(1-ssfreq)*ssfreq, ssfreq*ssfreq)

In [ ]:
clust_size1 = list(clust_ids[clust_sizes==1])


In [ ]:
frag_clust_size[:10]

In [ ]:
frag_size1 = fragments[frag_clust_size == 1]

In [ ]:
len(frag_size1)

In [ ]:
frag_size1_ss = (frag_clust_size == 1) & mask_state["ss"]

In [ ]:
sum(frag_size1_ss)

In [ ]:
frag_size1_ds = (frag_clust_size == 1) & mask_state["ds"]
sum(frag_size1_ds)

In [ ]:
sum(frag_clust_size == 1) 

In [ ]:
sum(mask_Rib & (frag_clust_size == 1))

In [ ]:
sum(mask_noRib & (frag_clust_size == 1))

In [ ]:
5721/13901

In [ ]:
methods = query(chaindata, chainschema, fragments, None, "method")


In [ ]:
method = methods[:,0]

In [ ]:
method[:10]

In [ ]:
xray = (method == "x-ray diffraction")

In [ ]:
sum(xray)

In [ ]:
nmr = (method == "solution nmr")

In [ ]:
sum(nmr)

In [ ]:
sum(nmr & (frag_clust_size == 1))

In [ ]:
sum((frag_clust_size == 1))

In [ ]:
4252/13901

In [ ]:
35455/len(fragments)

# Question 6
From what protein families do the fragments come from